In [1]:
# import json
# import gymnasium as gym
# from gym.spaces import Dict, Box
# import numpy as np

In [2]:
# from torch import nn
# from torch.functional import *

In [3]:
# import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from envs import BlendEnv, flatten_and_track_mappings

In [4]:
env = BlendEnv()

In [5]:
env = Monitor(env, "./logs")

In [6]:
env.reset()

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., 10., 30.,  0.,  0.],
       dtype=float32),
 {'dict_state': {'sources': {'s1': 0, 's2': 0},
   'blenders': {'j1': 0,
    'j2': 0,
    'j3': 0,
    'j4': 0,
    'j5': 0,
    'j6': 0,
    'j7': 0,
    'j8': 0},
   'demands': {'p1': 0, 'p2': 0},
   'properties': {'j1': {'q1': 0},
    'j2': {'q1': 0},
    'j3': {'q1': 0},
    'j4': {'q1': 0},
    'j5': {'q1': 0},
    'j6': {'q1': 0},
    'j7': {'q1': 0},
    'j8': {'q1': 0}},
   'sources_avail': {'s1': 10, 's2': 30},
   'demands_avail': {'p1': 0, 'p2': 0}}})

In [7]:
action_sample = {
    'source_blend':{
        's1': {'j1':1, 'j2':1, 'j3':1, 'j4':0}, # From s1 to b1, from s1 to b2 etc...
        's2': {'j1':1, 'j2':1, 'j3':0, 'j4':1},
    },
    
    # 'source_demand':{
    #     's1': {},
    #     's2': {}
    # },
    
    'blend_blend':{
        'j1': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j2': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j3': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j4': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j5': {},
        'j6': {},
        'j7': {},
        'j8': {}
    },
    
    'blend_demand':{
        'j1': {},
        'j2': {},
        'j3': {},
        'j4': {},
        'j5': {'p1':0, 'p2':0},
        'j6': {'p1':0, 'p2':0},
        'j7': {'p1':0, 'p2':0},
        'j8': {'p1':0, 'p2':0}
    },
    
    "tau": {"s1": 10, "s2": 10},
    
    "delta": {"p1": 0, "p2": 0}
}

In [8]:
# env._sanitize_action(action_sample)

In [9]:
# f, m = flatten_and_track_mappings(action_sample)

In [10]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log="./logs/")

Using cpu device
Wrapping the env in a DummyVecEnv.


In [11]:
env.reset()

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., 10., 30.,  0.,  0.],
       dtype=float32),
 {'dict_state': {'sources': {'s1': 0, 's2': 0},
   'blenders': {'j1': 0,
    'j2': 0,
    'j3': 0,
    'j4': 0,
    'j5': 0,
    'j6': 0,
    'j7': 0,
    'j8': 0},
   'demands': {'p1': 0, 'p2': 0},
   'properties': {'j1': {'q1': 0},
    'j2': {'q1': 0},
    'j3': {'q1': 0},
    'j4': {'q1': 0},
    'j5': {'q1': 0},
    'j6': {'q1': 0},
    'j7': {'q1': 0},
    'j8': {'q1': 0}},
   'sources_avail': {'s1': 10, 's2': 30},
   'demands_avail': {'p1': 0, 'p2': 0}}})

In [12]:
model.learn(total_timesteps=100000, log_interval=1)

Logging to ./logs/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6        |
|    ep_rew_mean     | 7.22e+03 |
| time/              |          |
|    fps             | 632      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6           |
|    ep_rew_mean          | 7.23e+03    |
| time/                   |             |
|    fps                  | 516         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015479516 |
|    clip_fraction        | 0.177       |
|    clip_range           | 0.2         |
|    entropy_loss         | -142        |
|    explained_variance   | 3.42e-05    |
|    l

In [ ]:
model.

In [ ]:
model.learn(total_timesteps=100)

model.save("ppo_myenv_model")

# Optionally, evaluate the trained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")